In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
import math
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
from configs import PATH_DATA_INPUT
from utils.optimization import get_gwg_interval_recommendations_from_features, get_gwg_interval_recommendations_from_features_2T
from utils.prediction import Predictor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 25)

In [4]:
patients_features = pd.read_csv(PATH_DATA_INPUT / "01_process.csv", index_col=0)
patients_features

,LGA,SGA,hipertension,diabetes,edad_a,paf_no_a,paf_diab_a,paf_tbcp_a,paf_hipe_a,paf_aler_a,pam_no_a,pam_diab_a,pam_tbcp_a,pam_hipe_a,pam_aler_a,pam_asma_a,pam_epil_a,pam_tiro_a,pam_card_a,pam_rena_a,paq_no_a,paq_cono_a,paq_miom_a,paq_infe_a,paq_liti_a,paq_cole_a,paq_apen_a,mecr_no_a,mecr_dur_a,mecr_des_a,mecr_nos_a,dro_no_a,dro_coca_a,dro_mari_a,dro_past_a,dro_amfe_a,dro_benz_a,epre_nor_a,epre_mal_a,epre_pree_a,epre_diab_a,epre_snc_a,epre_car_a,epre_dow_a,epre_otr_a,ae_a,ap_a,ppr_a,pter_a,mnat_a,mneo_a,hv_a,inicontp_a,nrocontr_a,IMC,GWG,activida_a_1,activida_a_2,activida_a_3,activida_a_4,activida_a_5,activida_a_6,activida_a_infrequent_sklearn,alcohol_a_2,alcohol_a_infrequent_sklearn,anticonc_a_3,anticonc_a_4,anticonc_a_5,anticonc_a_infrequent_sklearn,cigardia_a_2,cigardia_a_3,cigardia_a_infrequent_sklearn,comuna_a_la pintana,comuna_a_pirque,comuna_a_puente alto,comuna_a_san jose maipo,comuna_a_infrequent_sklearn,est_civil_a_1,est_civil_a_2,est_civil_a_3,est_civil_a_4,estnutri_a_1,estnutri_a_2,estnutri_a_3,estnutri_a_4,estudio_a_1,estudio_a_2,estudio_a_3,estudio_a_4,estudio_a_5,grpsanc_a_1,grpsanc_a_2,grpsanc_a_3,grpsanc_a_4,grpsanc_a_5,grpsanr_a_1,grpsanr_a_2,grpsanr_a_5,grpsanr_a_infrequent_sklearn,previsio_a_1,previsio_a_2,previsio_a_3,previsio_a_4,previsio_a_5,previsio_a_6
0,0,0,0,0,39.000000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,8.000000,9.000000,38.265306,12.108655,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,24.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,12.000000,6.000000,26.638918,9.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,28.000000,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,12.000000,12.000000,20.719787,15.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,0,0,0,0,24.000000,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,23.000000,6.000000,24.141519,17.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
4,1,0,0,0,25.000000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,22.000000,6.000000,20.446742,17.000000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93942,0,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.682786,37.443937,12.108655,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
93943,1,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.682786,37.443937,12.108655,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
93944,1,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.6

In [5]:
patients_features_sampled = patients_features.sample(int(len(patients_features)*0.3)).reset_index().drop(columns="index")
patients_features_sampled

,LGA,SGA,hipertension,diabetes,edad_a,paf_no_a,paf_diab_a,paf_tbcp_a,paf_hipe_a,paf_aler_a,pam_no_a,pam_diab_a,pam_tbcp_a,pam_hipe_a,pam_aler_a,pam_asma_a,pam_epil_a,pam_tiro_a,pam_card_a,pam_rena_a,paq_no_a,paq_cono_a,paq_miom_a,paq_infe_a,paq_liti_a,paq_cole_a,paq_apen_a,mecr_no_a,mecr_dur_a,mecr_des_a,mecr_nos_a,dro_no_a,dro_coca_a,dro_mari_a,dro_past_a,dro_amfe_a,dro_benz_a,epre_nor_a,epre_mal_a,epre_pree_a,epre_diab_a,epre_snc_a,epre_car_a,epre_dow_a,epre_otr_a,ae_a,ap_a,ppr_a,pter_a,mnat_a,mneo_a,hv_a,inicontp_a,nrocontr_a,IMC,GWG,activida_a_1,activida_a_2,activida_a_3,activida_a_4,activida_a_5,activida_a_6,activida_a_infrequent_sklearn,alcohol_a_2,alcohol_a_infrequent_sklearn,anticonc_a_3,anticonc_a_4,anticonc_a_5,anticonc_a_infrequent_sklearn,cigardia_a_2,cigardia_a_3,cigardia_a_infrequent_sklearn,comuna_a_la pintana,comuna_a_pirque,comuna_a_puente alto,comuna_a_san jose maipo,comuna_a_infrequent_sklearn,est_civil_a_1,est_civil_a_2,est_civil_a_3,est_civil_a_4,estnutri_a_1,estnutri_a_2,estnutri_a_3,estnutri_a_4,estudio_a_1,estudio_a_2,estudio_a_3,estudio_a_4,estudio_a_5,grpsanc_a_1,grpsanc_a_2,grpsanc_a_3,grpsanc_a_4,grpsanc_a_5,grpsanr_a_1,grpsanr_a_2,grpsanr_a_5,grpsanr_a_infrequent_sklearn,previsio_a_1,previsio_a_2,previsio_a_3,previsio_a_4,previsio_a_5,previsio_a_6
0,0,0,0,0,27.000000,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,7.000000,6.000000,28.698980,0.000000,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,27.223962,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1.00000,0.000000,0.068071,0.000000,0.000000,0.000000,1.000000,13.000000,3.000000,21.907582,10.000000,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,22.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1.00000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,10.000000,9.000000,27.688778,0.000000,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
3,1,0,0,0,27.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,15.000000,9.000000,25.208086,5.000000,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0
4,0,0,0,0,27.223962,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.20584,0.002913,0.068071,0.961308,0.009408,0.008558,1.004386,12.184671,7.682786,37.443937,12.108655,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28179,0,0,0,0,33.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.961308,0.000000,0.000000,3.000000,14.000000,6.000000,30.407627,5.000000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
28180,0,0,0,0,27.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,12.000000,29.136316,17.000000,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
28181,0,1,0,0,17.000000,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1.00000,0.000000,0.000000,0.000000,0.009408,0.008558,1.004386,8.000000,2.00000

In [6]:
def is_static_recomm(GWG, IMC):
    if IMC < 18.5:
        return 12 <= GWG <= 15
    elif IMC < 25:
        return 10 <= GWG <= 13
    elif IMC < 30:
        return 7 <= GWG <= 10
    else:
        return 6 <= GWG <= 7  
    
def IMC_label(IMC):
    if IMC < 18.5:
        return "bajo peso"
    elif IMC < 25:
        return "normal"
    elif IMC < 30:
        return "sobrepeso"
    else:
        return "obesidad"  

In [8]:
apprchs = {
    #"4T": [get_gwg_interval_recommendations_from_features, ["diabetes", "hipertension", "LGA", "SGA"]], 
           "2T": [get_gwg_interval_recommendations_from_features_2T, ["diab_hiper"]], 
     #      "1T": [get_gwg_interval_recommendations_from_features, ["IS_SICK"]]
}

recommendations_from_features_dict = {}

for app in apprchs:
    print(app)
    fo = apprchs[app][0]
    targets=apprchs[app][1]
    predictor=Predictor(targets=targets)
    if app == "2T":
        recommendations_from_features=fo(patients_features=patients_features_sampled, predictor=predictor, const_th1=0.4, const_th2=0.4,)
    else:
        recommendations_from_features=fo(patients_features=patients_features_sampled, predictor=predictor)
        
    recommendations_from_features_dict[app] = recommendations_from_features
    
    
    

2T
Total time 0:16:41.396374 to process 28184 obs


In [11]:
results = patients_features_sampled[["LGA", "SGA", "hipertension", "diabetes", "IMC", "GWG"]].copy()
results["enf"] = results.apply(lambda row: row[["LGA","SGA","hipertension","diabetes"]].max() , axis=1)
results["is_static_recomm"] = results.apply(lambda row:  is_static_recomm(GWG=row["GWG"], IMC=row["IMC"]), axis=1)
results["IMC_label"] = results["IMC"].apply(IMC_label)
for app in recommendations_from_features_dict:
    print(app)
    tbl = recommendations_from_features_dict[app].copy()
    tbl = tbl.join(results["GWG"])
    
    new_col = "{}_is_pers_recomm".format(app)
    
    display(tbl)
    tbl[new_col] = tbl.apply(lambda row: row["min_gwg_GWG"] <= row["GWG"] <= row["max_gwg_GWG"], axis=1)
    tbl[new_col] = tbl.apply(lambda row: None if math.isnan(row["best_gwg_score"]) else row[new_col], axis=1)
    
    results = results.join(tbl[[new_col]])

results

2T


,min_gwg_GWG,min_gwg_score,max_gwg_GWG,max_gwg_score,best_gwg_GWG,best_gwg_score,GWG
0,10.0,0.352890,16.0,0.362315,13.0,0.328769,0.000000
1,22.0,0.497074,25.0,0.443169,24.0,0.432984,10.000000
2,1.0,0.423299,7.0,0.426328,4.0,0.397016,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,5.000000
4,12.0,0.478498,18.0,0.497723,15.0,0.454103,12.108655
...,...,...,...,...,...,...,...
28179,12.0,0.329330,18.0,0.311121,15.0,0.297537,5.000000
28180,NaN,NaN,NaN,NaN,NaN,NaN,17.000000
28181,10.0,0.726613,12.0,0.743962,11.0,0.713150,6.000000
28182,14.0,0.326288,20.0,0.266518,17.0,0.247893,12.108655


,LGA,SGA,hipertension,diabetes,IMC,GWG,enf,is_static_recomm,IMC_label,2T_is_pers_recomm
0,0,0,0,0,28.698980,0.000000,0.0,False,sobrepeso,False
1,1,0,0,0,21.907582,10.000000,1.0,True,normal,False
2,0,0,0,0,27.688778,0.000000,0.0,False,sobrepeso,False
3,1,0,0,0,25.208086,5.000000,1.0,False,sobrepeso,None
4,0,0,0,0,37.443937,12.108655,0.0,False,obesidad,True
...,...,...,...,...,...,...,...,...,...,...
28179,0,0,0,0,30.407627,5.000000,0.0,False,obesidad,False
28180,0,0,0,0,29.136316,17.000000,0.0,False,sobrepeso,None
28181,0,1,0,0,19.477147,6.000000,1.0,False,normal,False
28182,0,0,0,0,51.674411,12.108655,0.0,False,obesidad,False


In [12]:
n = 1
finals_results = pd.DataFrame()
for app in apprchs:
    
    pers_recomm = "{}_is_pers_recomm".format(app)

    for agg_fx in ["mean", "count"]:
        
        cross_table_gen = pd.crosstab(index=results["is_static_recomm"],columns=results[pers_recomm], values=results["enf"], aggfunc=agg_fx)
        
        print("(n. {}) (general) {} - {}".format(n, app, agg_fx))
        n += 1
        
        display(cross_table_gen)
        
        for imc_label in ["bajo peso", "normal", "sobrepeso", "obesidad"]:
        
            mask = results["IMC_label"] == imc_label
            results_aux = results[mask].copy()
            print("(n. {}) {} - {} - {}".format(n, app, imc_label, agg_fx))
            n += 1

            
            crosstab = pd.crosstab(index=results_aux["is_static_recomm"],columns=results_aux[pers_recomm], values=results_aux["enf"], aggfunc=agg_fx)
            display(crosstab)

            
            

(n. 1) (general) 2T - mean


2T_is_pers_recomm,False,True
is_static_recomm,,
False,0.410270,0.321202
True,0.390545,0.322763


(n. 2) 2T - bajo peso - mean


2T_is_pers_recomm,False,True
is_static_recomm,,
False,0.421918,0.218182
True,0.397790,0.225806


(n. 3) 2T - normal - mean


...

2T_is_pers_recomm,False,True
is_static_recomm,,
False,6886,1255
True,2533,1310


(n. 9) 2T - sobrepeso - count


2T_is_pers_recomm,False,True
is_static_recomm,,
False,3679,1242
True,966,258


(n. 10) 2T - obesidad - count


2T_is_pers_recomm,False,True
is_static_recomm,,
False,3714,1542
True,212,136


In [13]:
print(results.isna().count())
results.isna().mean()


LGA                  28184
SGA                  28184
hipertension         28184
diabetes             28184
IMC                  28184
GWG                  28184
enf                  28184
is_static_recomm     28184
IMC_label            28184
2T_is_pers_recomm    28184
dtype: int64


LGA                  0.000000
SGA                  0.000000
hipertension         0.000000
diabetes             0.000000
IMC                  0.000000
GWG                  0.000000
enf                  0.000000
is_static_recomm     0.000000
IMC_label            0.000000
2T_is_pers_recomm    0.134402
dtype: float64